## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-05-35-00 +0000,wsj,June’s U.K. Home Buyer Demand Turns Positive f...,https://www.wsj.com/economy/central-banking/ju...
1,2025-07-10-05-27-30 +0000,nypost,Rock band with more than 1 million Spotify lis...,https://nypost.com/2025/07/10/us-news/rock-ban...
2,2025-07-10-05-23-26 +0000,nypost,Ex-California cop who allegedly partied at mus...,https://nypost.com/2025/07/10/us-news/californ...
3,2025-07-10-05-10-07 +0000,nypost,Massive haul of methamphetamine seized in one ...,https://nypost.com/2025/07/10/us-news/minnesot...
4,2025-07-10-05-03-24 +0000,nypost,Jealous ex-wife allegedly spied on former husb...,https://nypost.com/2025/07/10/us-news/californ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2296/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
140,trump,18
83,he,6
20,ex,6
223,tariffs,4
73,end,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,42
21,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,41
7,2025-07-10-04-39-28 +0000,nypost,Ex-CIA Director John Brennan says he’s had ‘no...,https://nypost.com/2025/07/10/us-news/ex-cia-d...,39
35,2025-07-09-20-49-26 +0000,nypost,Trump dismisses Iran threatening poolside Mar-...,https://nypost.com/2025/07/09/us-news/trump-sa...,39
27,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,37


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,42,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
7,39,2025-07-10-04-39-28 +0000,nypost,Ex-CIA Director John Brennan says he’s had ‘no...,https://nypost.com/2025/07/10/us-news/ex-cia-d...
6,22,2025-07-10-04-42-44 +0000,nypost,Philadelphia workers and city reach deal to en...,https://nypost.com/2025/07/10/us-news/philadel...
4,20,2025-07-10-05-03-24 +0000,nypost,Jealous ex-wife allegedly spied on former husb...,https://nypost.com/2025/07/10/us-news/californ...
36,18,2025-07-09-20-37-43 +0000,nypost,Mike Johnson steps up calls for fresh Russia s...,https://nypost.com/2025/07/09/us-news/mike-joh...
37,18,2025-07-09-20-25-00 +0000,wsj,Minutes from the Fed’s June policy meeting tea...,https://www.wsj.com/economy/central-banking/ju...
24,17,2025-07-10-00-41-46 +0000,nypost,Haunting photos show a glimpse into Camp Mysti...,https://nypost.com/2025/07/09/us-news/haunting...
34,16,2025-07-09-21-02-03 +0000,nypost,Ex-Long Island hospital CEO to sue parent comp...,https://nypost.com/2025/07/09/us-news/ousted-l...
65,16,2025-07-09-09-36-00 +0000,wsj,Malaysia’s Central Bank Cuts Rates for First T...,https://www.wsj.com/articles/malaysias-central...
35,15,2025-07-09-20-49-26 +0000,nypost,Trump dismisses Iran threatening poolside Mar-...,https://nypost.com/2025/07/09/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
